<a href="https://colab.research.google.com/github/sahilsudo/guvi-ds-t2-mongo/blob/main/students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports 
import pymongo
import pprint
import json
from pymongo import MongoClient

In [ ]:
#create a MongoClient instance
client = MongoClient("") 

In [ ]:
#create/connect to a database
student = client.student

In [ ]:
#create/connect to a collection
marks=student.marks

In [ ]:
#read the .json file
text=""
with open('students.json') as f:
    text=f.readlines()
list=[x for x in text]

In [ ]:
#insert documents from the file into collection
for i in list:
    d=json.loads(i)
    marks.insert_one(d)

In [ ]:
#get and print the collection
x=marks.find({})
for doc in x:
    pprint.pprint(doc)

In [ ]:
#Find the student name who scored maximum scores in all (exam, quiz and homework)?

x=marks.aggregate([
 {
    "$project": {
          "_id":"$_id",
          "name":"$name",
          "score":"$scores",
          "total_score":{"$sum":[{"$arrayElemAt": ["$scores.score", 0]},{"$arrayElemAt": ["$scores.score", 1]},{"$arrayElemAt": ["$scores.score", 2]}]}
    }

   },
    
  {
      "$sort":{"total_score":-1}
  },
    
   {"$limit":1} 
])

#print the result
for doc in x:
    pprint.pprint(doc)

In [ ]:
#Find students who scored below average in the exam and pass mark is 40%?

x=marks.aggregate([{"$group": {"_id":None, "sum_val":{"$avg":{"$arrayElemAt": ["$scores.score", 0]}}}}])
avg=0
for doc in x:
    avg=doc["sum_val"]

y=marks.aggregate([
    {
        "$match":{
            "$expr":{
                "$lt":[{"$arrayElemAt": ["$scores.score", 0]},avg]
            }
        }
    }
])

#print the result
for doc in y:
    pprint.pprint(doc)

In [ ]:
#Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all 
#the categories.

marks.aggregate( [
    {
           "$addFields": {
              "EPass":
                 {
                  "$cond":{"if": {"$gt":[{"$arrayElemAt": ["$scores.score", 0]},40]}, "then": "Pass", "else": "Fail"}
                 },
              "QPass":
                 {
                  "$cond":{"if": {"$gt":[{"$arrayElemAt": ["$scores.score", 1]},40]}, "then": "Pass", "else": "Fail"}
                 },
              "HPass":
                 {
                  "$cond":{"if": {"$gt":[{"$arrayElemAt": ["$scores.score", 2]},40]}, "then": "Pass", "else": "Fail"}
                 },
            
           }
         },
    
        {
         "$out": { "db": "student", "coll": "marks" }
        }
   ] )

#print the result
for doc in marks.find({}):
    pprint.pprint(doc)

In [ ]:
#Find the total and average of the exam, quiz and homework and store them in a
#separate collection.

marks4=student.marks4
marks.aggregate(pipeline= [
    {
           "$addFields": {
                "total":{
                    "$sum":[{"$arrayElemAt": ["$scores.score", 0]},{"$arrayElemAt": ["$scores.score", 1]},{"$arrayElemAt": ["$scores.score", 2]}]
                },
                "avg":{
                    "$avg":[{"$arrayElemAt": ["$scores.score", 0]},{"$arrayElemAt": ["$scores.score", 1]},{"$arrayElemAt": ["$scores.score", 2]}]
                }
           }
         },
    
        {
         "$out": { "db": "student", "coll": "marks4" }
        }
   ]) 

#print the result
for doc in marks4.find({}):
    pprint.pprint(doc)

In [ ]:
#Create a new collection which consists of students who scored below average and
#above 40% in all the categories.

marks5=student.marks5
x=marks.aggregate([{"$group": {"_id":None, "sum_val":{"$avg":{"$arrayElemAt": ["$scores.score", 0]}}}}])
y=marks.aggregate([{"$group": {"_id":None, "sum_val":{"$avg":{"$arrayElemAt": ["$scores.score", 1]}}}}])
z=marks.aggregate([{"$group": {"_id":None, "sum_val":{"$avg":{"$arrayElemAt": ["$scores.score", 2]}}}}])
avg1=0
avg2=0
abg3=0

for doc in x:
    avg1=doc["sum_val"]
for doc in y:
    avg2=doc["sum_val"]
for doc in z:
    avg3=doc["sum_val"]

y=marks.aggregate([
    {
                "$match":{
                    "$expr":{  "$and":
                                [{"$and":[{"$gt":[{"$arrayElemAt": ["$scores.score", 0]},40]},{"$lt":[{"$arrayElemAt": ["$scores.score", 0]},avg1]}]},
                                 {"$and":[{"$gt":[{"$arrayElemAt": ["$scores.score", 1]},40]},{"$lt":[{"$arrayElemAt": ["$scores.score", 1]},avg2]}]}, 
                                 {"$and":[{"$gt":[{"$arrayElemAt": ["$scores.score", 2]},40]},{"$lt":[{"$arrayElemAt": ["$scores.score", 2]},avg3]}]},        
                                ]
                            }
                        }
    },
    {
        "$project":{
            "_id":"$_id",
            "name":"$name",
            "score":"$scores"
        }
    },
    
    {
        "$out": { "db": "student", "coll": "marks5" }
    }

])

#print the result
for doc in marks5.find({}):
    pprint.pprint(doc)

In [ ]:
#Create a new collection which consists of students who scored below the fail mark
#in all the categories.

marks6=student.marks6
x=marks.aggregate([
    {
                "$match":{
                    "$expr":{  "$and":
                                [{"$lt":[{"$arrayElemAt": ["$scores.score", 0]},40]},
                                 {"$lt":[{"$arrayElemAt": ["$scores.score", 1]},40]}, 
                                 {"$lt":[{"$arrayElemAt": ["$scores.score", 2]},40]},        
                                ]
                            }
                        }
    },
    {
        "$project":{
            "_id":"$_id",
            "name":"$name",
            "score":"$scores"
        }
    },
    
    {
        "$out": { "db": "student", "coll": "marks6" }
    }

])

#print the result
for doc in marks6.find({}):
    pprint.pprint(doc)

In [ ]:
#Create a new collection which consists of students who scored above pass mark in
#all the categories.

marks7=student.marks7
x=marks.aggregate([
    {
                "$match":{
                    "$expr":{  "$and":
                                [{"$gt":[{"$arrayElemAt": ["$scores.score", 0]},40]},
                                 {"$gt":[{"$arrayElemAt": ["$scores.score", 1]},40]}, 
                                 {"$gt":[{"$arrayElemAt": ["$scores.score", 2]},40]},        
                                ]
                            }
                        }
    },
    {
        "$project":{
            "_id":"$_id",
            "name":"$name",
            "score":"$scores"
        }
    },
    
        
    {
        "$out": { "db": "student", "coll": "marks7" }
    }

])

#print the result
for doc in marks7.find({}):
    pprint.pprint(doc)

In [ ]:
#drop all collections
#marks.drop()
#marks4.drop()
#marks5.drop()
#marks6.drop()
#marks7.drop()